<a href="https://colab.research.google.com/github/MeDarina/GerMemeHate/blob/main/Classifier_GirlsDay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

girls-day-logo.avif
# Girls Day 2023 --  Wie verstehen Computer menschliche Sprache?


In dem folgenden Python-Code werden Filmkritiken in POSITIV oder NEGATIV eingeordnet.

Dazu werden zwei "klassische" Einordnungsmethoden - Naive Bayes und Support Vector Machines - verwendet, die beide nicht mehr state-of-the art ist, aber im Gegensatz zu diesen auch für Nicht-Spezialisten zumindest teilweise nachvollziehbar sind.


In der Statistik ist Einordnung bzw Klassifikation das Problem, zu identifizieren, zu welcher einer Reihe von Kategorien eine Beobachtung gehört. Beispiele aus der textuellen Datenverarbeitung sind 

*   Enthält eine Social Media Nachricht „Hate Speech“ oder nicht?
*   Handelt es sich bei einer E-Mail um einen Spam?
*   Ist eine Kundenbeurteilung positiv, negativ oder neutral?

## Bibliotheken und Daten laden

Zunächst müssen einige Bibliotheken geladen werden - einerseits Programmiercode, so dass Funktionen genutzt werden können, die bereits fertig gestellt wurden, anderseits um Daten zu nutzen.
Hier laden werden die Bibliotheken geladen, die wir für die Naive Bayes Methode brauchen (die anderen werden später an zugehöriger Stelle geladen)

In [ ]:
# Load and prepare the dataset
# To do so, after running the code, enter "d" (for Download) and download (you have to enter d each time before downloading a component)
# movie_reviews
# punkt
# averaged_perceptron_tagger
# stopwords
# After loading all needed components, enter "q" (for Quit)

import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> movie_reviews


      Package movie_reviews is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

Nachdem die Daten geladen wurden, müssen sie importiert werden. 
Danach werden sie in eine Liste geschrieben, in der eine Kritik mit der zugehörigen Einordnung (entweder Klasse "positiv" oder Klasse "negativ" geordnet wird.


In [ ]:
from nltk.corpus import movie_reviews
import random

documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

Hier können wir das 1. Element anschauen
Jedes Element besteht aus 2 Teilen: dem Text und der Zuordnung. Der Text ist eine Liste der Wörter und Satzzeichen (in der Reihenfolge wie sie im Text verwendet werden).

In [ ]:
print (documents[1])

(['everything', 'in', 'the', 'phantom', 'you', 'have', 'seen', 'many', 'times', 'before', 'and', 'there', 'is', 'nothing', 'new', 'presented', 'here', '.', 'wincer', 'displays', 'absolutely', 'no', 'skill', 'in', 'setting', 'up', 'an', 'exciting', 'action', 'sequence', '.', 'billy', 'zane', 'is', 'wooden', 'as', 'the', 'hero', '.', 'kristy', 'swanson', 'is', 'given', 'very', 'little', 'to', 'do', ',', 'and', 'does', 'very', 'little', 'with', 'it', '.', 'treat', 'williams', ',', 'looking', 'like', 'rhett', 'butler', 'but', 'sounding', 'like', 'mickey', 'mouse', ',', 'is', 'one', 'of', 'the', 'worst', 'villains', 'i', 'have', 'ever', 'seen', 'in', 'a', 'movie', '.', 'only', 'catherine', 'zeta', 'jones', ',', 'as', 'one', 'of', 'williams', 'cohorts', 'turns', 'in', 'a', 'good', 'performance', '.', 'she', 'has', 'energy', 'and', 'spunk', ',', 'which', 'the', 'movie', 'needed', 'much', 'more', 'of', '.', 'oh', 'yeah', ',', 'the', 'phantom', 'also', 'has', 'a', 'secret', 'identity', 'but', '

## Extraktion der Features

Die Zuordnung zu einer Klasse basiert oft auf beobachteten Mermalen zB Vorhandensein oder Nichtvorhandensein bestimmter beleidigender Worte, Rechtschreibfehler, ... .

Diese Merkmale nennt man auch Features und beide hier vorgestellten Methoden machen Gebrauch davon.

Hier laden wir eine Liste von positiven Wörtern und eine Liste von negativen Wörtern von einer Github-Datei. (Diese werden aufbereitet, so dass jedes Wort ein Element in einer Liste ist)

In [ ]:
# load positive and negative word list
import requests
def makeWordListFromURL(url):
  read_data = requests.get(url).content.decode(encoding="ISO-8859-1").rsplit()
  listOFWords= []
  i=0
  for word in read_data:
    if i>212:
      listOFWords.append(word)
      #print (i, ' ', l)
    i=i+1
  return listOFWords

positiveWords = makeWordListFromURL("https://raw.githubusercontent.com/shekhargulati/sentiment-analysis-python/master/opinion-lexicon-English/positive-words.txt")
negativeWords = makeWordListFromURL("https://raw.githubusercontent.com/shekhargulati/sentiment-analysis-python/master/opinion-lexicon-English/negative-words.txt")

Hier laden wir eine Liste von "stop words" oder auch "function words" genannt. Dies sind Wörter wie Artikel, Präpositionen, oder Personalprononomen - die im Gegensatz zu Nomen, Verben, Adjektiven / Adverben weniger zum Inhalt eines Textes beitragen.

In [ ]:
# load stopwords
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

Hier definieren wir eine Funktion, die alle Wörter in einem Dokument (einer Filmkritik) prüft und schaut, ob es in den vorher geladenen Listen (positive oder negative Wörter sowie der Stopwordliste) gelistet ist.
Außerdem schreibt diese Funktion jedem Wort seine Wortart (part of speech -> POS) zu.

In [ ]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))

def document_features(document):
    document_words = set(document)
    features = {}
    for word2 in document:
      features['pos({})'.format(word2)] = nltk.tag.pos_tag([word2])[0][1]
      features['isStopword({})'.format(word2)] = (word2 in stops)
      features['isPositive({})'.format(word2)] = (word2 in positiveWords)
      features['isNegative({})'.format(word2)] = (word2 in negativeWords)
    return features

Die Filmkritiken, die wir geladen haben, werden nun mit den Merkmalen / Eigenschaften versehen, um die Wahrscheinlichkeiten berechnen zu können.

In [ ]:

featuresets = [(document_features(d), c) for (d,c) in documents[:1000]]


Die klassifizierten und mit Markmalen / Eigenschaften versehenen Daten werden in Trainings - und Testset aufgeteilt, um einerseits den Klassifizierer zu trainieren und ihn dann zu evaluieren (auf ungesehenen Daten)

In [ ]:
train_set, test_set = featuresets[800:], featuresets[:200]

In [ ]:
print (featuresets)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Einordnung von Filmkritiken mit Naive Bayes

[Naive Bayes](https://https://en.wikipedia.org/wiki/Naive_Bayes_classifier) ist ein Algorithmus der maschinellen Textverarbeitung, der verwendet wird, um Texte automatisch zu klassifizieren.
Im Falle von Textklassifikation wird Naive Bayes verwendet, um die Wahrscheinlichkeit zu berechnen, dass ein bestimmtes Dokument zu einer bestimmten Kategorie gehört.
Der Algorithmus analysiert den Text und extrahiert bestimmte Merkmale - in diesem Beispiel die Wortart aller Wörter sowie das das Vorhandensein / Nichtvorhandensein bestimmter Wörter - die mit bestimmten Kategorien verbunden sind.
Anschließend werden diese Merkmale verwendet, um die Wahrscheinlichkeit zu berechnen, dass das Dokument zu jeder Kategorie gehört.
Der Algorithmus wählt dann die Kategorie aus, die die höchste Wahrscheinlichkeit hat, und klassifiziert das Dokument entsprechend.

In [ ]:
# Train Naive Bayes classifier
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

## Evaluation der Naive Bayes Methode

Man sollte berechnen können, wie gut eine Klassifikation ist (Evaluation)
Dafür braucht man ein ungesehenes Set mit klassifizierten Daten (Test-Set)
Die einfachste Metrik ist ”%-Agreement” (Übereinstimmung) d.h. wie viele Vorhersagen waren richtig?

In [ ]:
# Test the classifier
print(nltk.classify.accuracy(nb_classifier, test_set))

0.635


Wir können uns hier die hilfreichsten Features ausgeben lassen

In [ ]:
# Show the most important features as interpreted by Naive Bayes
nb_classifier.show_most_informative_features(10)

Most Informative Features
      isNegative(chosen) = False             neg : pos    =      7.7 : 1.0
      isPositive(chosen) = False             neg : pos    =      7.7 : 1.0
      isStopword(chosen) = False             neg : pos    =      7.7 : 1.0
             pos(chosen) = 'NN'              neg : pos    =      7.7 : 1.0
       isNegative(awful) = True              neg : pos    =      7.0 : 1.0
        isNegative(joke) = True              neg : pos    =      7.0 : 1.0
      isNegative(lawyer) = False             pos : neg    =      7.0 : 1.0
    isNegative(produced) = False             neg : pos    =      7.0 : 1.0
    isNegative(talented) = False             neg : pos    =      7.0 : 1.0
       isPositive(awful) = False             neg : pos    =      7.0 : 1.0


Hier können wir die Methode auf einem selbstgeschrieben Text ausprobieren

In [ ]:
#from nltk.tokenize import word_tokenize
s = 'The movie was so outstanding, I was totally fascinated. Just great and wonderful and magic.'
nb_classifier.classify(document_features(s))

'neg'

## Einordnung mit SVM

Wie auch Naive Bayes sind SVMs (Support Vector Machines) ist ein Algorithmus der maschinellen Textverarbeitung, der verwendet wird, um Texte automatisch zu klassifizieren.
Der Algorithmus sucht nach einem optimalen Trennpunkt zwischen den verschiedenen Kategorien, indem er versucht, eine sogenannte "Margin" zu maximieren - das ist der Abstand zwischen den Datenpunkten der verschiedenen Kategorien.
SVMs wählen dann den Trennpunkt, der diese Margin am besten maximiert.

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC
from sklearn import model_selection

In [ ]:
svc = SklearnClassifier(SVC(kernel = 'linear'))
svc.train(train_set)

Evaluation der SVM-Methode

In [ ]:
print (nltk.classify.accuracy(svc, test_set))

0.77
